In [ ]:
import numpy as np
import pandas as pd 
import os 
import matplotlib.pyplot as plt 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df =pd.read_csv("data/prices.csv", header=0)
df

In [ ]:
#size of the dataset in 851264 rows and 7 columns
print(df.shape)

In [ ]:
# takes 20 different and unique from symbol
df.symbol.unique()[0:20]

In [ ]:

print(len(df.symbol.values))

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.date.unique()

In [ ]:
comp_info = pd.read_csv('data/securities.csv')
comp_info

In [ ]:
comp_info["Ticker symbol"].nunique()

In [ ]:
comp_info.loc[comp_info.Security.str.startswith('Face') , :]

In [ ]:
comp_plot = comp_info.loc[(comp_info["Security"] == 'Yahoo Inc.') | (comp_info["Security"] == 'Xerox Corp.') | (comp_info["Security"] == 'Adobe Systems Inc')
              | (comp_info["Security"] == 'Microsoft Corp.') | (comp_info["Security"] == 'Adobe Systems Inc') 
              | (comp_info["Security"] == 'Facebook') | (comp_info["Security"] == 'Goldman Sachs Group') , ["Ticker symbol"] ]["Ticker symbol"] 
print(comp_plot)

In [ ]:

def plotter(code):
    global closing_stock ,opening_stock
    f, axs = plt.subplots(2,2,figsize=(8,8))
    plt.subplot(212)
    company = df[df['symbol']==code]
    company = company.open.values.astype('float32')
    company = company.reshape(-1, 1)
    opening_stock = company

    
    plt.grid(True)
    plt.xlabel('Time') 
    plt.ylabel(code + " open stock prices") 
    plt.title('prices Vs Time')
    plt.plot(company , 'g')
    
    plt.subplot(211)
    
    company_close = df[df['symbol']==code]
    
    company_close = company_close.close.values.astype('float32')
    
    company_close = company_close.reshape(-1, 1)
    
    closing_stock = company_close

    
    plt.xlabel('Time') 
    plt.ylabel(code + " close stock prices")
    plt.title('prices Vs Time') 
    plt.grid(True) 
    plt.plot(company_close , 'b')
    plt.show() 

for i in comp_plot:
    plotter(i)

In [ ]:
closing_stock[:,0]

In [ ]:
stocks = closing_stock[: , 0]
print(stocks)
stocks = stocks.reshape(len(stocks) , 1)

**Feature scaling the vector for better model performance.**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) 
stocks = scaler.fit_transform(stocks) 

In [ ]:
train = int(len(stocks) * 0.80)
test = len(stocks) - train 

In [ ]:

print(train , test)

In [ ]:
train = stocks[0:train]
print(train)

In [ ]:
test = stocks[len(train) : ]

In [ ]:
train = train.reshape(len(train) , 1)
test = test.reshape(len(test) , 1)

In [ ]:
print(train.shape , test.shape)

In [ ]:
def process_data(data , n_features):
    dataX, dataY = [], [] 
    for i in range(len(data)-n_features-1):
        a = data[i:(i+n_features), 0]
        dataX.append(a) 
        dataY.append(data[i + n_features, 0])
    return np.array(dataX), np.array(dataY)


In [ ]:
n_features = 2
trainX, trainY = process_data(train, n_features)
testX, testY = process_data(test, n_features)

In [ ]:
print(trainX.shape , trainY.shape , testX.shape , testY.shape)

In [ ]:
trainX = trainX.reshape(trainX.shape[0] , 1 ,trainX.shape[1])
testX = testX.reshape(testX.shape[0] , 1 ,testX.shape[1])

In [ ]:
import math 
from keras.models import Sequential 
from keras.layers import Dense , BatchNormalization , Dropout , Activation 
from keras.layers import LSTM , GRU 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error 
from keras.optimizers import Adam , SGD , RMSprop 

In [ ]:
filepath="model/stock_weights.hdf5"
from keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, epsilon=0.0001, patience=1, verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [ ]:
model = Sequential()
model.add(GRU(256 , input_shape = (1 , n_features) , return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(256))
model.add(Dropout(0.4))
model.add(Dense(64 ,  activation = 'relu'))
model.add(Dense(1))
print(model.summary())

In [ ]:
model.compile(loss='mean_squared_error', optimizer=Adam(lr = 0.0005) , metrics = ['mean_squared_error'])

In [ ]:
 
history = model.fit(trainX, trainY, epochs=100 , batch_size = 128 , 
          callbacks = [checkpoint , lr_reduce] , validation_data = (testX,testY))    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['mean_squared_error']) 
plt.plot(history.history['val_mean_squared_error'])
plt.title('model mean squared error')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left') 
plt.show() 

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss') 
plt.xlabel('epoch') 
plt.legend(['train', 'test'], loc='upper left')  
plt.show() 

In [ ]:

def model_score(model, X_train, y_train, X_test, y_test):
   
    trainScore = model.evaluate(X_train, y_train, verbose=0)
 
    print('Train Score: %.5f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))
    testScore = model.evaluate(X_test, y_test, verbose=0)
    print('Test Score: %.5f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    return trainScore[0], testScore[0]

model_score(model, trainX, trainY , testX, testY)

In [ ]:
pred = model.predict(testX)
pred = scaler.inverse_transform(pred)
pred[:10]


In [ ]:
testY = testY.reshape(testY.shape[0] , 1)
testY = scaler.inverse_transform(testY)
testY[:10]

In [ ]:
print("Red - Predicted Stock Prices  ,  Blue - Actual Stock Prices")
plt.rcParams["figure.figsize"] = (15,7)
plt.plot(testY , 'b')
plt.plot(pred , 'r')
plt.xlabel('Time')
plt.ylabel('Stock Prices')
plt.title('Check the accuracy of the model with time')
plt.grid(True)
plt.show()